# Lec 12 - Trees

For this notebook, images are either cited from CS61A slides lecture12, or screenshots of lecture videos. <br>
[Link to lecture slides](http://inst.eecs.berkeley.edu/~cs61a/sp18/assets/slides/12-Trees_1pp.pdf) <br>
[Link to lecture video](https://www.youtube.com/watch?v=If99StVLChY&list=PL6BsET-8jgYU_D4zfjxpO_ItfvMimn4Yv&index=1)

## Box and Pointer Notation
A way to represent lists within our environment diagrams.
<img src='box_pointer.png' width='750'>

### Closure
- **Closure Property** The result of combination can itself be combined using the same method.
- Closure is powerful because it permits us to create hierachical structures.
- Hierachical structures are made up of parts, which themselves are made up of parts. - Quite powerful!

Example - **List**: List can contain list as elements (in addition to anything else).

## Slicing

In [1]:
odds = [1,3,5,7,9]
odds

[1, 3, 5, 7, 9]

In [2]:
[odds[i] for i in range(1,3)]

[3, 5]

Slicing is a more convinient way for doing this. It has the same rule as `range`: doesn't include ending.

**Note: Slicing creates new values.**

In [3]:
odds[1:3]

[3, 5]

In [4]:
odds[1:], odds[:3], odds[:]

([3, 5, 7, 9], [1, 3, 5], [1, 3, 5, 7, 9])

## Processing Container Values

**Sequence Aggregation** <br>
Built-in functions take in iterable arguments and aggregate them into a value. <br><br>
**Example 1** <br>
`sum(iterable[, start])` -> value <br>
(Note here the `[]` is not true square brackets -- It's an expression in python documents implying the second argument is optional)

In [5]:
sum(odds, 0), sum(odds, 10)

(25, 35)

Why argument `start` is useful

In [6]:
try:
    print(sum([[2,3], [4]], [])) # start = []
    print(sum([[2,3], [4]]))     # no start values: start = 0 -- TypeError raised
except TypeError as e:
    print(e)

[2, 3, 4]
unsupported operand type(s) for +: 'int' and 'list'


In [7]:
[2,3] + [4]

[2, 3, 4]

In [8]:
try:
    [2,3] + 0
except TypeError as e:
    print(e)

can only concatenate list (not "int") to list


**Example 2** <br>
`max(iterable[, key=func])` -> value (with a single iterable argument) <br>
`max(a,b,c,...[,key=func])` -> value (with two or more arguments)

In [9]:
max(range(5)), max(0,1,2,3,4)

(4, 4)

In [10]:
max(range(10), key=lambda x: 7-(x-4)*(x-2))

3

In [11]:
(lambda x: 7-(x-4)*(x-2))(3)

8

Note the `max` returns value of the arguments (`x`=3) not the key function of arguments (`f(x)`=8) ! <br><br>
**Example 3** <br>
`all(iterable)` -> bool <br>
Return `True` if `bool(x)` is `True` for all `x` in the iterable argument.<br>
If the iterable argument is empty, returns `True`.

In [12]:
bool(-1), bool(5), bool(0)

(True, True, False)

In [13]:
bool('Hello World!'), bool('')

(True, False)

In [14]:
all([x < 5 for x in range(5)])

True

In [15]:
all(range(5))

False

## Tree
How to describe a tree structure

<img src='tree_abstraction.png' width='800'>

## Implementing Tree Abstraction

In [16]:
def tree(label, branches=[]):
    for branch in branches:
        assert is_tree(branch), 'branches must be trees!'
    return [label] + list(branches)

In [17]:
def branches(tree):
    return tree[1:]

In [18]:
def label(tree):
    return tree[0]

In [19]:
def is_tree(tree):
    if type(tree) != list or len(tree) < 1:
        return False
    for branch in branches(tree):
        if not is_tree(branch):
            return False
    return True

In [20]:
def is_leaf(tree):
    return not branches(tree)

<img src='implement_tree_abstraction.png' width='300'>

In [21]:
try:
    print(tree(3, [tree(5)]))
    print(tree(3, [5]))
except AssertionError as e:
    print(e)

[3, [5]]
branches must be trees!


In [22]:
tree(3, [tree(1), tree(2, [tree(1), tree(1)])])

[3, [1], [2, [1], [1]]]

In [23]:
t = tree(1, [tree(5, [tree(7)]), tree(6)])
t

[1, [5, [7]], [6]]

In [24]:
label(t)

1

In [25]:
branches(t)

[[5, [7]], [6]]

In [26]:
branches(t)[0]  ## the index-0 branch (like the left suntree)

[5, [7]]

In [27]:
is_tree(branches(t)[0])

True

In [28]:
label(branches(t)[0])

5

## Tree Processing
Functions take in tree or ouput tree are often tree recursions themelves.

### Tree Processing uses Recursion
- Processing the leaf is often the base case of processing a tree. <br>
- The recursive case typically <font color='red'>makes a recursive call on each branch, then aggregates.</font>

In [29]:
def count_leaves(t):
    """Count the leaves of a tree."""
    if is_leaf(t):
        return 1
    else:
        branch_counts = [count_leaves(b) for b in branches(t)]
        return sum(branch_counts)

In [30]:
def leaves(tree):
    """Return a list containg leaf labels of the tree."""
    if is_leaf(tree):
        return [label(tree)] # sum for lists
    else:
        branch_labels = [leaves(b) for b in branches(tree)]
        return sum(branch_labels, [])  # flatten a nested list!

### Example - Fibonacci Tree
<img src='fib_tree.png' width='600'>

In [31]:
def fib_tree(n):
    if n <= 1:
        return tree(n)
    else:
        left, right = fib_tree(n-2), fib_tree(n-1)
        return tree(label(left)+label(right), [left, right])

In [32]:
fib_tree(5)

[5, [2, [1], [1, [0], [1]]], [3, [1, [0], [1]], [2, [1], [1, [0], [1]]]]]

In [33]:
count_leaves(fib_tree(5))

8

In [34]:
leaves(fib_tree(5))

[1, 0, 1, 0, 1, 1, 0, 1]

### Creating Trees
A function that **creates a tree from another tree** is typically also **recursive**.

In [35]:
def increment_leaves(t):
    """Return a tree like t but with all leaf labels incremented."""
    if is_leaf(t):
        return tree(label(t)+1)
    else:
        bs = [increment_leaves(b) for b in branches(t)]
        return tree(label(t), bs)

In [36]:
def increment(t):
    """Return a tree like t but with all labels incremented."""
    return tree(label(t), [increment(t) for b in branches(t)])

For the second function, We don't need an independent base case for leaves, because when `t` is a leaf, `branches(t)`=`[]`, which is also included in the general case

### Printing Trees
Another example for tree processing.

In [37]:
def print_tree(t):
    """Print a tree (without return values)."""
    print(label(t))
    for b in branches(t):
        print_tree(b)

In [38]:
print_tree(fib_tree(4))

3
1
0
1
2
1
1
0
1


In [39]:
t

[1, [5, [7]], [6]]

In [40]:
print_tree(t)

1
5
7
6


We can see all the labels in the tree from `print_tree()`, but we cannot see the structure. Here is a better function using indentation.

In [41]:
def print_tree(t, indent=0):
    print('  '*indent + str(label(t)))
    for b in branches(t):
        print_tree(b, indent+1)

In [42]:
print_tree(fib_tree(4))

3
  1
    0
    1
  2
    1
    1
      0
      1


In [43]:
t

[1, [5, [7]], [6]]

In [44]:
print_tree(t)

1
  5
    7
  6
